<a href="https://colab.research.google.com/github/DawnKelvin/AI-Future/blob/main/RecyclableImgClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Environment Setup
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os


# Dataset: TrashNet
A popular open-source dataset with ~2,500 labeled images (glass, paper, cardboard, plastic, metal, trash). Created by Stanford students, it's widely used for lightweight waste classification. Includes resized, ready-to-use versions.

Data: https://github.com/garythung/trashnet.git

In [3]:
# Load and preprocess dataset
# Define image size and batch size
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Assuming you uploaded dataset in folders: plastic/, glass/, paper/, etc.
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/dataset-resized',  # Adjust path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/dataset-resized',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 2029 images belonging to 6 classes.
Found 505 images belonging to 6 classes.


In [4]:
# Define lightweight CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=IMG_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 826,854 (3.15 MB)

 Trainable params: 826,854 (3.15 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3897 - loss: 1.4545

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 639s 10s/step - accuracy: 0.3902 - loss: 1.4535 - val_accuracy: 0.4099 - val_loss: 1.4847
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 55s 848ms/step - accuracy: 0.5103 - loss: 1.2166 - val_accuracy: 0.4475 - val_loss: 1.4004
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 51s 795ms/step - accuracy: 0.5454 - loss: 1.1237 - val_accuracy: 0.4495 - val_loss: 1.3709
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 51s 790ms/step - accuracy: 0.6429 - loss: 0.9073 - val_accuracy: 0.4693 - val_loss: 1.3833
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 85s 834ms/step - accuracy: 0.6809 - loss: 0.8114 - val_accuracy: 0.4792 - val_loss: 1.3175


This code defines and compiles a lightweight Convolutional Neural Network (CNN) using TensorFlow's Keras API.

Here's a breakdown:

1.  **`model = tf.keras.models.Sequential([...])`**: This line initializes a sequential model, which means the layers are stacked one after another.

2.  **Convolutional Layers (`Conv2D`)**:
    *   `tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=IMG_SIZE + (3,))`: This is the first convolutional layer.
        *   `16`: The number of filters (feature detectors) in this layer.
        *   `(3,3)`: The size of the convolution kernel (the window that slides over the image).
        *   `activation='relu'`: The ReLU (Rectified Linear Unit) activation function is applied to the output of the convolution.
        *   `input_shape=IMG_SIZE + (3,)`: Specifies the expected input shape. `IMG_SIZE` is a tuple `(height, width)`, and `(3,)` adds the color channels (for RGB images).
    *   `tf.keras.layers.Conv2D(32, (3,3), activation='relu')`: A second convolutional layer with 32 filters.
    *   `tf.keras.layers.Conv2D(64, (3,3), activation='relu')`: A third convolutional layer with 64 filters.

3.  **MaxPooling Layers (`MaxPooling2D`)**:
    *   `tf.keras.layers.MaxPooling2D(2,2)`: These layers reduce the spatial dimensions (width and height) of the feature maps. A pool size of `(2,2)` means the layer takes the maximum value from each 2x2 window.

4.  **Flatten Layer (`Flatten`)**:
    *   `tf.keras.layers.Flatten()`: This layer converts the 2D feature maps into a 1D vector, which is necessary to feed the data into the dense layers.

5.  **Dense Layers (`Dense`)**:
    *   `tf.keras.layers.Dense(64, activation='relu')`: A fully connected layer with 64 neurons and ReLU activation.
    *   `tf.keras.layers.Dense(train_data.num_classes, activation='softmax')`: The output layer. The number of neurons is equal to the number of classes in your dataset (`train_data.num_classes`), and the `softmax` activation function outputs probabilities for each class, summing up to 1.

6.  **Compile the Model**:
    *   `model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])`: This configures the model for training.
        *   `optimizer='adam'`: The optimization algorithm used to update the model's weights during training.
        *   `loss='categorical_crossentropy'`: The loss function suitable for multi-class classification problems with one-hot encoded labels.
        *   `metrics=['accuracy']`: The metric used to evaluate the model's performance during training and evaluation.

7.  **Model Summary**:
    *   `model.summary()`: Prints a summary of the model's architecture, including the output shape and the number of parameters for each layer.

This code defines the structure of the neural network that will be trained to classify images of different recyclable materials.

In [7]:
# Evaluate model and save
model.evaluate(val_data)
model.save('recyclable_classifier.h5')


16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 285ms/step - accuracy: 0.4749 - loss: 1.3159


In [9]:
# Convert to TensorFlow Lite for Rasperry Pi
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpclhjhb21'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  139092497510288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497511440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497509712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497507984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497510864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497510480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497508176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497508560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497512976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139092497513936: TensorSpec(shape=(), dtype=tf.resource, name=None)


# Test on sample dataset

# Task
Convert the model to TensorFlow Lite and test it on a sample dataset.

## Load the tflite model

### Subtask:
Load the saved `.tflite` model into a TensorFlow Lite Interpreter.


**Reasoning**:
Import the necessary class and load the TFLite model using the Interpreter.



In [11]:
from tensorflow.lite.python.interpreter import Interpreter

# Load the TFLite model and allocate tensors.
interpreter = Interpreter(model_path='recyclable_classifier.tflite')
interpreter.allocate_tensors()

## Prepare sample data

### Subtask:
Load and preprocess a sample image or a batch of images to match the input requirements of the TFLite model.


**Reasoning**:
Load and preprocess a sample image to match the input requirements of the TFLite model.



In [12]:
# Choose a sample image file path (replace with an actual path from your dataset)
sample_image_path = '/content/drive/MyDrive/Colab Notebooks/dataset-resized/plastic/plastic1.jpg' # Example path

# Load the image
img = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=IMG_SIZE)

# Convert the image to a NumPy array
img_array = tf.keras.preprocessing.image.img_to_array(img)

# Expand dimensions to include batch size
img_array = tf.expand_dims(img_array, 0) # Create a batch

# Rescale the image
img_array = img_array / 255.0

print("Sample image loaded and preprocessed with shape:", img_array.shape)

Sample image loaded and preprocessed with shape: (1, 128, 128, 3)


## Run inference

### Subtask:
Use the TFLite Interpreter to run inference on the prepared sample data.


**Reasoning**:
Use the TFLite interpreter to run inference on the prepared sample data by getting input and output tensor details, setting the input tensor, invoking the interpreter, and getting the output tensor.



In [13]:
# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], img_array)

# Invoke the interpreter
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Inference completed. Output data shape:", output_data.shape)
print("Output data:", output_data)

Inference completed. Output data shape: (1, 6)
Output data: [[0.00244701 0.3235134  0.11538694 0.46402404 0.07492646 0.01970217]]


## Interpret the results

### Subtask:
Process the output of the inference to get the class predictions and their probabilities.


**Reasoning**:
Process the output of the inference to get the class predictions and their probabilities by getting the predicted class index using argmax, retrieving class labels, creating a list of class names, finding the predicted class name and its probability, and printing the results.



In [14]:
import numpy as np

# Get the index of the highest probability
predicted_class_index = np.argmax(output_data)

# Get the class labels from train_data.class_indices
class_indices = train_data.class_indices

# Create a list of class names in the correct order
class_names = [key for key, value in sorted(class_indices.items(), key=lambda item: item[1])]

# Get the predicted class name
predicted_class_name = class_names[predicted_class_index]

# Get the probability of the predicted class
predicted_probability = output_data[0][predicted_class_index]

# Print the predicted class name and its probability
print("Predicted class:", predicted_class_name)
print("Predicted probability:", predicted_probability)

Predicted class: paper
Predicted probability: 0.46402404


## Summary:

### Data Analysis Key Findings

*   The TensorFlow Lite model was successfully loaded using the `Interpreter` class and `allocate_tensors()` was called to prepare it for inference.
*   A sample image was loaded, resized to 128x128 pixels, converted to a NumPy array, expanded to include a batch dimension (resulting shape (1, 128, 128, 3)), and rescaled to the range \[0, 1].
*   Inference was successfully run on the prepared sample data using the TFLite interpreter. The output shape was (1, 6), indicating a batch size of 1 and 6 output values (likely class probabilities).
*   The predicted class for the sample image was identified as "paper" with a probability of approximately 0.464.

### Insights or Next Steps

*   To fully test the model, run inference on a larger and more diverse sample dataset and evaluate metrics like accuracy, precision, and recall.
*   Compare the performance of the TFLite model with the original TensorFlow model on the same test set to ensure conversion did not significantly impact accuracy.
